In [1]:
import torch
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [2]:
spdata = pd.read_pickle('SP.pkl')
mpdata = pd.read_pickle('MP.pkl')

In [3]:
# 指定标签

spdata['label'] = 0
mpdata['label'] = 1

In [4]:
# 提取共有特征

intersection_columns = list(set(spdata.columns).intersection(set(mpdata.columns)))

In [5]:
# 合并单双原发癌表

mixdata = pd.DataFrame()
# mixdata = mixdata.append(spdata[intersection_columns].sample(n=8288, random_state=1))
mixdata = mixdata.append(spdata[intersection_columns])
mixdata = mixdata.append(mpdata[mpdata['Record number recode']==1][intersection_columns])

In [6]:
mixdata.columns

Index([&#39;Radiation sequence with surgery&#39;, &#39;Survival months&#39;,
       &#39;Age at diagnosis&#39;, &#39;ER Status Recode Breast Cancer (1990+)&#39;,
       &#39;Derived AJCC M, 7th ed (2010-2015)&#39;, &#39;Derived SEER Combined M (2016+)&#39;,
       &#39;SEER Combined Mets at DX-liver (2010+)&#39;, &#39;Histologic Type ICD-O-3&#39;,
       &#39;Breast Subtype (2010+)&#39;, &#39;Derived HER2 Recode (2010+)&#39;,
       &#39;Derived SEER Combined N (2016+)&#39;, &#39;RX Summ--Surg Prim Site (1998+)&#39;,
       &#39;Derived AJCC T, 7th ed (2010-2015)&#39;, &#39;Primary Site - labeled&#39;,
       &#39;Sequence number&#39;, &#39;Derived SEER Cmb Stg Grp (2016+)&#39;,
       &#39;SEER Combined Mets at DX-bone (2010+)&#39;, &#39;CS tumor size (2004-2015)&#39;,
       &#39;Grade&#39;, &#39;Patient ID&#39;, &#39;Race recode (W, B, AI, API)&#39;,
       &#39;SEER Combined Mets at DX-brain (2010+)&#39;, &#39;Radiation recode&#39;,
       &#39;PR Status Recode Breast Cancer (1990+

In [7]:
# 指定标签及特征

label = mixdata['label']
features = mixdata[['Radiation sequence with surgery', 'Survival months',
       'Age at diagnosis', 'ER Status Recode Breast Cancer (1990+)',
       'Derived AJCC M, 7th ed (2010-2015)', 'Derived SEER Combined M (2016+)',
       'SEER Combined Mets at DX-liver (2010+)', 'Histologic Type ICD-O-3',
       'Breast Subtype (2010+)', 'Derived HER2 Recode (2010+)',
       'Derived SEER Combined N (2016+)', 'RX Summ--Surg Prim Site (1998+)',
       'Derived AJCC T, 7th ed (2010-2015)', 'Primary Site - labeled',
       'Sequence number', 'Derived SEER Cmb Stg Grp (2016+)',
       'SEER Combined Mets at DX-bone (2010+)', 'CS tumor size (2004-2015)',
       'Grade', 'Patient ID', 'Race recode (W, B, AI, API)',
       'SEER Combined Mets at DX-brain (2010+)', 'Radiation recode',
       'PR Status Recode Breast Cancer (1990+)', 'Marital status at diagnosis',
       'Laterality', 'Chemotherapy recode (yes, no/unk)', 'label',
       'COD to site recode', 'Derived SEER Combined T (2016+)',
       'Tumor Size Summary (2016+)', 'Site recode ICD-O-3/WHO 2008',
       'Derived AJCC N, 7th ed (2010-2015)', 'Year of diagnosis',
       'CS site-specific factor 7 (2004+ varying by schema)',
       'SEER Combined Mets at DX-lung (2010+)',
       'Derived AJCC Stage Group, 7th ed (2010-2015)']]

In [8]:
# One-Hot 编码分类数据

features = pd.get_dummies(features)

In [9]:
# 数值标准化

features = preprocessing.StandardScaler().fit_transform(features)

In [10]:
# 指定 x, y

x = torch.from_numpy(np.array(features)).float()
y = torch.from_numpy(np.array(label))

In [11]:
x.shape

torch.Size([323435, 845])

In [12]:
# 构建网络

net = torch.nn.Sequential(
    torch.nn.Linear(845, 2000),
    torch.nn.ReLU(),
    torch.nn.Linear(2000, 2)
)

optimizer = torch.optim.SGD(net.parameters(), lr=0.5)
loss_func = torch.nn.CrossEntropyLoss()

In [13]:
for t in range(100):
    prediction = net(x)
    loss = loss_func(prediction, y)

    print(t, loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 tensor(0.7312, grad_fn=&lt;NllLossBackward&gt;)
1 tensor(1.1195, grad_fn=&lt;NllLossBackward&gt;)
2 tensor(0.8644, grad_fn=&lt;NllLossBackward&gt;)
3 tensor(0.6920, grad_fn=&lt;NllLossBackward&gt;)


KeyboardInterrupt: 